# The financial portfolio case

### Initialization 

In [1]:
import numpy as np

In [2]:
# Definimos los parámetros fundamentales de nuestro modelo
risk_aversion = 1.0   # Risk aversion en el enfoque mean-variance
rho = 0.1             # Multiplicador de Lagrange para el término de costes
A = 1.0               # Multiplicador del término de normalización en el problema QUBO
n_digits = 4          # Numero de dígitos que utilizaremos para describir el peso de cada asset
n_assets = 14         # Número de assets que consideramos
n_intervals = 2       # Número de instantes de tiempo
n_variables = n_assets * n_digits * n_intervals # Número de variables del problema
print('numero de variables del problema: ', n_variables)

numero de variables del problema:  112


So, there are 3 types of variables:

{digit_0, digit_1, digit_2, digit_3 }<br><br>
{asset_0, asset_1, ...,  asset_13} <br><br>
{time_0, time_1} 


In [3]:
J = {}
h = {}
bias = 0.0

In [4]:
# Vectores de medias de retornos
means = np.random.randn(n_intervals, n_assets)

### 1) Returns contribution

In [5]:
# Este bucle calcula la contribución de la media de los retornos
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] = 2**(-digit) * -means[time, asset]


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(time, asset, digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(time, asset, digit) = \frac{-1}{{2}^{\text{digit }}} \text{means}(time, asset)$

`Q_l.interactions( (i1, j1, k1)x(i1, j1, k1) ) = f(i1,j1,k1)`

In [6]:
time = []

for _ in range(n_intervals):
    time.append(_)




digit = []

for _ in range(n_digits):
    digit.append(_)

    
asset = []

for _ in range(n_assets):
    asset.append(_) 



t = time
y = asset
z = digit


from collections import defaultdict

x_l = defaultdict(lambda: 0.)  # user variable interactions
x_q = defaultdict(lambda: 0.)  # user variable interactions


Q_l = defaultdict(lambda: 0.)
Q_q = defaultdict(lambda: 0.)

for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x_l[(i1,i2,j1,j2,k1,k2)] = (-1 * 2 ** - k1) * means[i1,j1]  # k1 = k2 due the "if k1!=k2: continue"  
                            Q_l[index_i] = (-1 * 2 ** - k1) * means[i1,j1]   # and the same for i and j  
                    

print("this is the input dictionary for DWave:")
                
#print(Q_l)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x_l)
    

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Normalization contribution

In [7]:
# Contribución a bias de la normalización (no es necesaria)
bias += A

### Normalization linear contribution

In [8]:
# Este bucle lleva la contribución lineal de la normalización
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] += A * (2**(-digit) - 1) * 2**(-digit)           


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(digit) = \left ( \frac{1}{{2}^{\text{digit }}}-1 \right )\frac{A}{{2}^{\text{digit }}} $

`Q_l.interactions( (i1, j1, k1)x(i1, j1, k1) ) = f(k1)`

In [9]:
for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x_l[(i1,i2,j1,j2,k1,k2)] += A * (2**(-k1) - 1) * 2**(-k1)    
                            Q_l[index_i] += A * (2**(-k1) - 1) * 2**(-k1)     
                    

print("this is the input dictionary for DWave:")
                
#print(Q_l)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x_l)
    

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


### Normalization quadratic contribution

In [10]:
# Este bucle lleva la contribucion cuadratica de la normalizacion
for time in range(n_intervals):
    for asset1 in range(n_assets):
        for digit1 in range(n_digits):
            for asset2 in range(n_assets):
                for digit2 in range(n_digits):
                    if asset1 >= asset2 and digit1 >= digit2: continue
                    coord1 = time*n_digits*n_assets + asset1*n_digits + digit1
                    coord2 = time*n_digits*n_assets + asset2*n_digits + digit2
                    J[(coord1,coord2)] = A * 2**(-digit1) * 2**(-digit2)




<br>
<br>
 
# WARNING!!! 
 
<br>

The following code block:

<br>

`if asset1 >= asset2: continue` <br>
`if digit1 >= digit2: continue`

<br>

transpiles in a mathematical expression as below: 

<br>

$\sum_{\text{time}}\sum_{\text{asset2>asset1}}\sum_{\text{digit2>digit1}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2}$

<br>
<br>

$f(digit1, digit2) = \frac{A}{{2}^{\text{digit1 + digit2 }}} $


<br>
<br>

**However**, the code block of the problem is:

<br>

`if asset1 >= asset2 and digit1 >= digit2: continue`

<br>

transpiling as the following mathematical expression:

<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset2>asset1}}\sum_{\text{digit1}}\sum_{\text{digit2}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2} \\
+ \\
\sum_{\text{time}}\sum_{\text{asset1>=asset2}}\sum_{\text{digit2>digit1}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2}$

In [11]:


#### the case of    if asset1 >= asset2: continue
#                   if digit1 >= digit2: continue`

"""
for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1>=j2: continue
                            if k1>=k2: continue
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            x[(i1,i2,j1,j2,k1,k2)] += A * 2**(-k1) * 2**(-k2)  
                            
                            
                             
                            # converting into a upper triangular matrix
                            if index_i > index_j:
                                Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            else:
                                Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
print(Q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
print(x)

"""

'\nfor i1 in t:\n        for j1 in y:\n            for k1 in z:\n                for i2 in t:\n                    for j2 in y:\n                        for k2 in z:\n                            if i1!=i2: continue\n                            if j1>=j2: continue\n                            if k1>=k2: continue\n                            index_i = i1*len(y)*len(z) + j1*len(z) + k1\n                            index_j = i2*len(y)*len(z) + j2*len(z) + k2\n                            \n                            x[(i1,i2,j1,j2,k1,k2)] += A * 2**(-k1) * 2**(-k2)  \n                            \n                            \n                             \n                            # converting into a upper triangular matrix\n                            if index_i > index_j:\n                                Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  \n                            else:\n                                Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     \n        

<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset2>asset1}}\sum_{\text{digit1}}\sum_{\text{digit2}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2} \\
+ \\
\sum_{\text{time}}\sum_{\text{asset1>=asset2}}\sum_{\text{digit2>digit1}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2}$


<br>
<br>


`Q_q.interactions( (i1, j1, k1)x(i1, j2, k2) | not(j1>=j2 and k1>=k2) ) = f(k1,k2)`

In [12]:
#### the case of      if asset1 >= asset2 and digit1 >= digit2: continue

for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1>=j2 and k1>=k2: continue
                                
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            x_q[(i1,i2,j1,j2,k1,k2)] += A * 2**(-k1) * 2**(-k2)  
                            
                            Q_q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)
                            
                             
                            # converting into a upper triangular matrix
                            #if index_i > index_j:
                            #    Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            #else:
                            #    Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
# print(Q_q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
#print(x_q)



this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


###  Covariance linear contribution

In [13]:
# Matrices de covarianza de retornos
covs = np.random.rand(n_intervals, n_assets, n_assets)

In [14]:
# Este bucle lleva la contribución lineal de la covarianza
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] += covs[time, asset, asset] * 2**(-2.0*digit)

# print(h)


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(time,asset,digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(time, asset,digit) =  \frac{1}{{2}^{2\text{digit }}}\text{covs[time, asset, asset]} $

`Q_l.interactions( (i1, j1, k1)x(i1, j1, k1) ) = f(i1,j1,k1)`

In [15]:
for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x_l[(i1,i2,j1,j2,k1,k2)] += covs[i1, j1, j1] * 2**(-2.0*k1)   
                            Q_l[index_i] += covs[i1, j1, j1] * 2**(-2.0*k1)     
                    

print("this is the input dictionary for DWave:")
                
#print(Q_l)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x_l)
    

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


## Covariance quadratic contribution

In [16]:
# Este bucle lleva la contribución cuadratica de la covarianza
for time in range(n_intervals):
    for asset1 in range(n_assets):
        for digit1 in range(n_digits):
            for asset2 in range(n_assets):
                for digit2 in range(n_digits):
                    if asset1 >= asset2 and digit1 >= digit2: continue
                    coeficiente = 0.5 * risk_aversion * covs[time, asset1, asset2] * 2**(-digit1) * 2**(-digit2)
                    coord1 = time*n_digits*n_assets + asset1*n_digits + digit1
                    coord2 = time*n_digits*n_assets + asset2*n_digits + digit2
                    J[(coord1, coord2)] += coeficiente
                    

**Again**, the code block of the problem ...

<br>

`if asset1 >= asset2 and digit1 >= digit2: continue`

<br>

... is transpiling as the following mathematical expression:

<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset2>asset1}}\sum_{\text{digit1}}\sum_{\text{digit2}}f(time, asset1, asset2,digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2} \\
+ \\
\sum_{\text{time}}\sum_{\text{asset1>=asset2}}\sum_{\text{digit2>digit1}}f(time, asset1, asset2,digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2}$

<br>
<br>

$f(time, asset1, asset2,digit1,digit2) = \frac{1}{2\cdot2^{digit1+digit2}}\text{risk_aversion}\cdot \text{covs[time, asset1, asset2]}$

`Q_q.interactions( (i1, j1, k1)x(i1, j2, k2) | not(j1>=j2 and k1>=k2) ) = f(i,j1,j2)`

In [17]:
for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1>=j2 and k1>=k2: continue
                                
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            coeficiente = 0.5 * risk_aversion * covs[i1, j1, j2] * 2**(-k1) * 2**(-k2)
                            
                            x_q[(i1,i2,j1,j2,k1,k2)] += coeficiente  
                            
                            Q_q[(index_i, index_j)] += coeficiente
                            
                             
                            # converting into a upper triangular matrix
                            #if index_i > index_j:
                            #    Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            #else:
                            #    Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
#print(Q_q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
#print(x_q)



this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Linear cost contribution

In [18]:
# Vector de costes de transaccion
costs = np.array([0.01]*n_assets)

In [19]:
# Este bucle lleva la contribución lineal de los costes
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] += 2**(-2.0*digit) * rho * costs[asset]
            

<br>
<br>
<br>


$ \sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(asset,digit)\text{    }\text{X}_{time, asset, digit} $

<br>
<br>

$f(asset,digit) =  \frac{\text{rho}}{{2}^{2\text{digit }}}\text{cost[asset]} $

`Q_l.interactions( (i1, j1, k1)x(i1, j1, k1) ) = f(j1,k1)`

In [20]:
for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x_l[(i1,i2,j1,j2,k1,k2)] += 2**(-2.0*k1) * rho * costs[j1]  
                            Q_l[index_i]  += 2**(-2.0*k1) * rho * costs[j1]     
                    

print("this is the input dictionary for DWave:")
                
#print(Q_l)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x_l)

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Quadratic cost contribution  (intra assets)

In [21]:
# Este bucle lleva la contribución cuadrática de los costes (INTRA ASSET)
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit1 in range(n_digits):
            for digit2 in range(n_digits):
                if digit1 >= digit2: continue
                coord1 = time*n_digits*n_assets + asset*n_digits + digit1
                coord2 = time*n_digits*n_assets + asset*n_digits + digit2
                J[(coord1, coord2)] += 2**(-digit1) * 2**(-digit2) * rho * costs[asset]
                

<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit2>digit1}}f(digit1,digit2)\text{    }\text{X}_{time, asset, digit1}\text{X}_{time, asset, digit2}$

<br>
<br>


$f(asset, digit1, digit2) = \frac{\text{rho}}{{2}^{\text{digit1 + digit2} } } \text{costs[asset]} $

`Q_q.interactions( (i1, j1, k1)x(i1, j1, k2>k1) ) = f(j1,k1,k2)`

In [22]:
for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue 
                            if k1>=k2: continue
                                
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            
                            x_q[(i1,i2,j1,j2,k1,k2)] += 2**(-k1) * 2**(-k2) * rho * costs[j1]  
                            
                            Q_q[(index_i, index_j)]  += 2**(-k1) * 2**(-k2) * rho * costs[j1]
                            
                             
                            # converting into a upper triangular matrix
                            #if index_i > index_j:
                            #    Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            #else:
                            #    Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
#print(Q_q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
#print(x_q)

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Quadratic cost contribution ( inter assets )

<br>
<br>
<br>

### Por favor, observen que hay una incoherencia en el código que pasó BBVA

<br>

En la siguiente línea ...

<br>

`coord2 = (time+1) * n_digits*n_assets + asset*n_digits + digit2`

<br>

`time+1` genera una matriz más grande que la que corresponde a las 112 variables del problema 

In [23]:
# Este bucle lleva la contribución cuadrática de los costes (INTER ASSET)
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit1 in range(n_digits):
            for digit2 in range(n_digits):
                coord1 = time     * n_digits*n_assets + asset*n_digits + digit1
                coord2 = (time+1) * n_digits*n_assets + asset*n_digits + digit2
                J[(coord1, coord2)] = -2**(-digit1) * 2**(-digit2) * rho * costs[asset]
                


<br>

$\sum_{\text{time1}}\sum_{\text{time2=time1+1}}\sum_{\text{asset}}\sum_{\text{digit1}}\sum_{\text{digit2}}f(asset, digit1,digit2)\text{    }\text{X}_{time1, asset, digit1}\text{X}_{time2, asset, digit2}$

<br>
<br>

$f(asset, digit1, digit2) = \frac{\text{rho}}{{2}^{\text{digit1 + digit2} } } \text{costs[asset]} $

`Q_q.interactions( (i1, j1, k1)x(i2=i1+1, j1, k2) ) = f(j1,k1,k2)`

In [24]:
for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            i2 = i1+1  # BBVA condition (it is not compatible with the number of variables) 
                            #if i1+1!=i2: continue   # my version (coherent with the number of variables)
                            if j1!=j2: continue 
                            #if k1>=k2: continue
                                
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            
                            x_q[(i1,i2,j1,j2,k1,k2)] = -2**(-k1) * 2**(-k2) * rho * costs[j1] 
                            
                            Q_q[(index_i, index_j)]  = -2**(-k1) * 2**(-k2) * rho * costs[j1]
                            
                             
                            # converting into a upper triangular matrix
                            #if index_i > index_j:
                            #    Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            #else:
                            #    Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
#print(Q_q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
#print(x_q)

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Resolution of the QUBO problem

In [28]:
import dimod

# Se resuelve el modelo de Ising
model = dimod.BinaryQuadraticModel(h, J, bias, dimod.BINARY)

print(model)

sampler = dimod.SimulatedAnnealingSampler()
response_1 = sampler.sample(model, beta_range=[1/3.0, 1/0.05], num_sweeps=100, num_reads=10)
#response = sampler.sample(model, beta_range=[1/3.0, 1/0.05], num_sweeps=3000, num_reads=300)


print(response_1.first)



BinaryQuadraticModel({0: -0.9939145119243729, 1: -0.7825037058660478, 2: -0.46263846540898923, 3: -0.24916588582348595, 4: 1.0460013288660646, 5: 0.07596337907049147, 6: -0.07377763180538949, 7: -0.06482864623785355, 8: 1.5275659035720695, 9: 0.38730264479378185, 10: 0.0995312456488277, 11: 0.026235603637398047, 12: -5.037847123673476e-05, 13: -0.3517780443180427, 14: -0.2638272359296274, 15: -0.15389817140746526, 16: 1.7425751508193983, 17: 0.4317037731602854, 18: 0.10595593601778924, 19: 0.025503980368306263, 20: -0.9966982045979729, 21: -0.7899058975513178, 22: -0.46784214758874165, 23: -0.25214337349764154, 24: 0.3445872755650109, 25: -0.3239658527288218, 26: -0.28604779899224264, 27: -0.1740401176530793, 28: 1.1620165221675551, 29: 0.09662613483125534, 30: -0.0727824641475029, 31: -0.06666511496453409, 32: -0.8960408617931392, 33: -0.7750367576969307, 34: -0.46927246054855554, 35: -0.2550747506993003, 36: 3.117128767257085, 37: 1.1936080721952471, 38: 0.5055649582392995, 39: 0.229

Sample(sample={0: 0, 1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 1, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 1, 94: 1, 95: 1, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 1, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 1, 113: 1, 114: 1, 115: 1, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 1, 127: 0, 128: 0, 129: 1, 130: 0, 131: 0, 132: 0, 133: 0, 134: 1, 135: 0, 136: 0,

In [27]:
# Se resuelve el modelo de Ising
model = dimod.BinaryQuadraticModel(Q_l, Q_q, bias, dimod.BINARY)

print(model)

sampler = dimod.SimulatedAnnealingSampler()
response_2 = sampler.sample(model, beta_range=[1/3.0, 1/0.05], num_sweeps=100, num_reads=10)
#response = sampler.sample(model, beta_range=[1/3.0, 1/0.05], num_sweeps=3000, num_reads=300)

print(response_2.first)




BinaryQuadraticModel({0: -0.9939145119243729, 1: -0.7825037058660478, 2: -0.46263846540898923, 3: -0.24916588582348595, 4: 1.0460013288660646, 5: 0.07596337907049147, 6: -0.07377763180538949, 7: -0.06482864623785355, 8: 1.5275659035720695, 9: 0.38730264479378185, 10: 0.0995312456488277, 11: 0.026235603637398047, 12: -5.037847123673476e-05, 13: -0.3517780443180427, 14: -0.2638272359296274, 15: -0.15389817140746526, 16: 1.7425751508193983, 17: 0.4317037731602854, 18: 0.10595593601778924, 19: 0.025503980368306263, 20: -0.9966982045979729, 21: -0.7899058975513178, 22: -0.46784214758874165, 23: -0.25214337349764154, 24: 0.3445872755650109, 25: -0.3239658527288218, 26: -0.28604779899224264, 27: -0.1740401176530793, 28: 1.1620165221675551, 29: 0.09662613483125534, 30: -0.0727824641475029, 31: -0.06666511496453409, 32: -0.8960408617931392, 33: -0.7750367576969307, 34: -0.46927246054855554, 35: -0.2550747506993003, 36: 3.117128767257085, 37: 1.1936080721952471, 38: 0.5055649582392995, 39: 0.229

Sample(sample={0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 1, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 1, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 1, 94: 1, 95: 1, 96: 0, 97: 0, 98: 0, 99: 1, 100: 0, 101: 0, 102: 0, 103: 1, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 1, 114: 0, 115: 0, 116: 1, 117: 0, 118: 1, 119: 1, 120: 1, 121: 0, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 0, 130: 0, 131: 0, 132: 1, 133: 1, 134: 0, 135: 1, 136: 0,